In [1]:
import pandas as pd
import numpy as np

## TO DO
Fix the total savings products caused by adding 1 to random number function   
Fix the total savings balance  
Add probability distribution to closed accounts to make high numbers less likely

### Fetch names and save locally

In [ ]:
# top 100 male and female names in the 2010s from the social security administration
first_names_page = pd.read_html('https://www.ssa.gov/oact/babynames/decades/names2010s.html')

# wikipedia page for popular last names in multiple countries in North America
#last_names_page = pd.read_html('https://en.wikipedia.org/wiki/List_of_most_common_surnames_in_North_America')

# Census with top 1000 American surnames in 2010
last_names_table = pd.read_excel('https://www2.census.gov/topics/genealogy/2010surnames/Names_2010Census_Top1000.xlsx')

In [ ]:
first_name_table = first_names_page[0]
#last_names_table = last_names_page[8] # uncomment if you're using the wikipedia page

In [ ]:
first_names_cols = ['rank', 'male_name', 'male_number', 'female_name', 'female_number']
first_name_table.columns = first_names_cols
first_name_table = first_name_table[:200]

In [ ]:
males = first_name_table['male_name']
females = first_name_table['female_name']
all_names = males.append(females)

In [ ]:
last_names = last_names_table.iloc[2:1000,0]
lln=list(last_names)
for i,x in enumerate(lln):
    lln[i] = x.lower().capitalize()
clean_last_names = pd.DataFrame(lln)

In [ ]:
clean_last_names.to_csv('last_names.csv', index = False, header=True)
all_names.to_csv('first_names.csv', index = False, header = True)

### Membership_participant

In [2]:
def make_participation_type(portion_101, total_individuals_desired):
    # probability of 101, not portion of 101
    from numpy.random import choice
    participation_types = [101, 102]
    portion_102 = 1-portion_101
    participation_probability = [portion_101, portion_102]
    participation_type = choice(participation_types, total_individuals_desired, p=participation_probability )
    return(list(participation_type))

def make_participation_type_table(participation_type_list):
    participation_type_list = list(participation_type_list)
    n_primaries = len([x for x in participation_type_list if x == 101])
    n_secondaries = len(participation_type_list)-n_primaries
    member_numbers = list(np.random.randint(1000,9000, n_primaries))
    account_type = [101]*len(member_numbers)
    secondary_account_type = [102]*n_secondaries
    account_type += secondary_account_type
    secondary_accounts = list(np.random.choice(member_numbers, n_secondaries, replace = True))
    member_numbers += secondary_accounts
    
    #make random individual ids 
    individual_id = list(np.random.randint(100,10000,len(participation_type_list)))
    
    #make final dataframe
    membership_participation = pd.DataFrame(member_numbers, columns = ['member_nbr'])
    membership_participation['participation_type']=account_type
    membership_participation['individual_id']=individual_id
    
    return (membership_participation)

def make_more_primary_memberships(membership_participation_df, portion_new_memberships):
    n = int(len(membership_participation_df) * portion_new_memberships)
    inds = list(membership_participation_df['individual_id'])
    duplicate_individuals = list(np.random.choice(inds, n, replace=True))
    new_memberships = list(np.random.randint(1000,9000, n))
    participation_type = [101]*n
    more_memberships = pd.DataFrame(new_memberships, columns = ['member_nbr'])
    more_memberships['participation_type']=participation_type
    more_memberships['individual_id']=duplicate_individuals
    
    membership_participation_df = membership_participation_df.append(more_memberships).reset_index(drop=True)
    
    return( membership_participation_df)


### Individual Table

In [ ]:
fn = pd.read_csv('first_names.csv')
ln = pd.read_csv('last_names.csv')
fn = list(fn['0'])
ln = list(ln['0'])

In [3]:
def load_name_lists():
    try:
        fn = pd.read_csv('first_names.csv')
        ln = pd.read_csv('last_names.csv')
        fn = list(fn['0'])
        ln = list(ln['0'])
    # add steps to download names
    except:
        return('error in loading names')
        #fn = pd.read_csv()
        #ln = pd.read_csv()
    return(fn, ln)

def make_dates(length):
    import datetime
    start = datetime.date(2000,1,1)
    end = datetime.date.today()
    dates = [np.random.random() * (end-start)+ start for _ in range(length)] #https://stackoverflow.com/questions/50165501/generate-random-list-of-timestamps-in-python
    for i, x in enumerate(dates):
        dates[i]=str(x)
    return(dates)

def make_name_key(length):
    first_name_key = list(np.random.randint(0,400,length))
    last_name_key = list(np.random.randint(0,1000,length))
    return(first_name_key, last_name_key)

def generate_names(first_name_key, last_name_key):
    fn, ln = load_name_lists()
    
    for i,x in enumerate(first_name_key):
        first_name_key[i]=fn[x]
    
    for i, x in enumerate(last_name_key):
        last_name_key[i]=ln[x]
    
    names = pd.DataFrame(first_name_key, columns = ['first_name'])
    names['last_name']=last_name_key
    
    return(names)

def get_individual_ids(membership_participation_table):
    individual_ids = set(membership_participation_table['individual_id'])
    return(individual_ids)

def create_individual_table(membership_participation_table):
    # read in the membership participation table to get the individual ids and determine how many names to generate
    individual_ids = get_individual_ids(membership_participation_table)
    n = len(individual_ids)
    
    #name keys
    fnk, lnk = make_name_key(n)
    
    #names
    individual = generate_names(fnk, lnk)
    
    #add dates
    open_date = make_dates(n) #add function to ensure open date is suitable time after DOB
    #dob = make_dates(n)
    
    #add attributes to names
    #individual['dob']=dob
    individual['open_date']=open_date
    individual['individual_id']=individual_ids
    
    return(individual)

### agr_membertotal_today

In [4]:
import random

values = list(np.linspace(1,90,90))
probabilities = list(np.linspace(1,100,90))

def agr_membertotal_row(member_number):
    row = []
    row.append(member_number)
    
    #first four columns = 1-4
    cls_ln_cnt = np.random.randint(0,5)
    cls_sv_cnt = np.random.randint(0,5)
    opn_ln_all_cnt = np.random.randint(0,5)
    opn_sv_all_cnt = np.random.randint(1,5)
    
    #open loan balance 5
    open_loan_bal = sum(np.random.randint(1000,500000,opn_ln_all_cnt))
    
    #open saving balance 6
    open_sv_bal = sum(np.random.randint(25,100000,opn_sv_all_cnt))
    
    #dividend year to date 7
    div_ytd_amt = open_sv_bal * .01
    
    #interest year to date 8
    int_ytd_amt = open_loan_bal * .05
    
    #savings share deposit balance 9
    SV_SD_BAL = np.random.randint(25,open_sv_bal)

    #savings share deposit count 10
    SV_SD_CNT = np.random.randint(1,opn_sv_all_cnt+1)
    
    #savings certificate balance 11
    SV_CERT_BAL = open_sv_bal - SV_SD_BAL  
    
    #savings certificate count 12
    SV_CERT_CNT = opn_sv_all_cnt - SV_SD_CNT
    
    #max days delinquent 13
    if cls_ln_cnt > 0:
        max_days_delnq_cnt = random.choices(values, cum_weights=probabilities, k=1)[0]
    else:
        max_days_delnq_cnt = 0
    
    #max months delq 14
    max_mths_delnq_cnt = round(int(max_days_delnq_cnt)/30)
    
    #written off loan count 15
    if max_days_delnq_cnt>89:
        wo_ln_cnt = cls_ln_cnt * np.random.randint(1,5)
    else:
        wo_ln_cnt = 0
        
    #written off loan balance 16
    if wo_ln_cnt > 0:
        wo_ln_bal = wo_ln_cnt * np.random.randint(1000,500000)
    else:
        wo_ln_bal = 0
        
    columns = (
    cls_ln_cnt,
    cls_sv_cnt,
    opn_ln_all_cnt,
    opn_sv_all_cnt,
    open_loan_bal,
    open_sv_bal,
    div_ytd_amt,
    int_ytd_amt,
    SV_SD_BAL,
    SV_SD_CNT,
    SV_CERT_BAL,
    SV_CERT_CNT,
    max_days_delnq_cnt,
    max_mths_delnq_cnt,
    wo_ln_cnt,
    wo_ln_bal      
    )
    
    for variable in columns:
        row.append(variable)
    
    return(row)

def generate_agr_membertotal(list_of_member_numbers): 
    col_headers = [
    'member_nbr',
    'cls_ln_cnt',
    'cls_sv_cnt',
    'opn_ln_all_cnt',
    'opn_sv_all_cnt',
    'open_loan_bal',
    'open_sv_bal',
    'div_ytd_amt',
    'int_ytd_amt',
    'sv_sd_bal',
    'sv_sd_cnt',
    'sv_cert_bal',
    'sv_cert_cnt',
    'max_days_delnq_cnt',
    'max_mths_delnq_cnt',
    'wo_ln_cnt',
    'wo_ln_bal'] 
    
    data = []
    
    for member_nbr in list_of_member_numbers:
        data.append(agr_membertotal_row(member_nbr))
    
    data = pd.DataFrame(data, columns = col_headers)
    
    return(data)

In [22]:
#participation type table
# input (portion to be 101, total individuals desired)
ptype = make_participation_type(.5, 100)
mp = make_participation_type_table(ptype)

#add more members with shared accounts
# input (probability an individual gets a primary membership)
membershipparticipant_today = make_more_primary_memberships(mp, .5)

#create individual table
individual_today = create_individual_table(participation_type)

#create agr_membertotal table
member_nbr = list(participation_type['member_nbr'])
agr_membertotal_today = generate_agr_membertotal(member_nbr)

In [23]:
membershipparticipant_today

,member_nbr,participation_type,individual_id
0,2058,101,2495
1,7265,101,4855
2,1748,101,4114
3,2965,101,1606
4,7760,101,6297
...,...,...,...
145,6898,101,2874
146,8434,101,4741
147,1383,101,6761
148,2959,101,4515


In [24]:
individual_today

,first_name,last_name,open_date,individual_id
0,Paige,Boyle,2003-09-26,9224
1,Skylar,Roberts,2000-09-04,3084
2,Declan,Mcguire,2010-11-19,4115
3,Alexa,Mccall,2001-08-01,4121
4,Joseph,Dixon,2014-11-19,5664
...,...,...,...,...
95,Sophia,Bartlett,2015-03-14,4069
96,Aurora,Wilkins,2018-08-12,2534
97,Elena,Meyers,2001-09-28,4078
98,Kaylee,House,2010-10-08,1015


In [25]:
agr_membertotal_today

,member_nbr,cls_ln_cnt,cls_sv_cnt,opn_ln_all_cnt,opn_sv_all_cnt,open_loan_bal,open_sv_bal,div_ytd_amt,int_ytd_amt,sv_sd_bal,sv_sd_cnt,sv_cert_bal,sv_cert_cnt,max_days_delnq_cnt,max_mths_delnq_cnt,wo_ln_cnt,wo_ln_bal
0,6644,0,3,0,2,0,108153,1081.53,0.00,77844,1,30309,1,0.0,0,0,0
1,4684,3,3,4,3,961654,125594,1255.94,48082.70,70991,2,54603,1,79.0,3,0,0
2,8708,3,1,4,4,1345499,179554,1795.54,67274.95,13809,1,165745,3,90.0,3,6,1354362
3,2464,3,2,2,1,557674,27074,270.74,27883.70,175,1,26899,0,52.0,2,0,0
4,4364,3,0,3,1,717492,25087,250.87,35874.60,21828,1,3259,0,14.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,5219,0,1,4,2,620146,114277,1142.77,31007.30,56854,1,57423,1,0.0,0,0,0
146,7116,3,2,3,1,848319,76063,760.63,42415.95,2532,1,73531,0,12.0,0,0,0
147,4883,3,3,0,3,0,129786,1297.86,0.00,121050,1,8736,2,78.0,3,0,0
148,3406,0,2,4,3,759943,199552,1995.52,37997.15,92341,3,107211,0,0.0,0,0,0


### Export results to sqlite database

In [26]:
import sqlite3
from sqlite3 import Error

conn = sqlite3.connect('demo_data.db')
participation_type.to_sql('membershipparticipant_today', conn, if_exists='replace', index = False)
individual.to_sql('individual_today', conn, if_exists = 'replace', index = False)
agr_membertotal_today.to_sql('agr_membertotal_today', conn, if_exists = 'replace', index = False)

In [ ]:
#return random name
r = np.random.randint(0,10)
r2 =r+1
fnk, lnk = make_name_key(10)
names = generate_names(fnk, lnk)
names[r:r2]

### Notes

In [ ]:
https://www.newyorkfed.org/microeconomics/databank
    
Debt type	Average balance
Mortgage debt	$35,660
Home equity lines of credit	$1,370
Auto loan	$4,760
Credit card debt	$2,980
Student loan debt	$5,610
Other debt (personal loans, payday loans, etc.)	$1,520

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

mu, sigma = 6.1, 1.3 # mean and standard deviation

s = np.random.lognormal(mu, sigma, 1000)
count, bins, ignored = plt.hist(s, 100, density=True, align='mid')

x = np.linspace(min(bins), max(bins), 10000)

pdf = (np.exp(-(np.log(x) - mu)**2 / (2 * sigma**2))

       / (x * sigma * np.sqrt(2 * np.pi)))

plt.plot(x, pdf, linewidth=2, color='r')

plt.axis('tight')

plt.show()

In [ ]:
mu, sigma = 6.1, 1.35 # mean and standard deviation

s = np.random.lognormal(mu, sigma, 1000)
s.mean(), s.max()

In [ ]:
test = generate_agr_membertotal([12345, 6789])

In [ ]:
test

Determine row wise

#### Savings distributions
Balance
* Minimum 25  
* Maximum 250,000  
* Mean  7,000  

Count
* Minimum 1
* Maximum 5
* Mean 2.5

#### Loan distributions

Balance  
* Minimum 0  
* Maximum 1,000,000  
* Mean 92,000  

Count  
* Minimum 0  
* Maximum 5  
* Mean 


#### Logic for populating fields

Random
CLS_LN_CNT = 0-5  
CLS_SV_CNT = 0-5  

OPN_LN_ALL_CNT = 0-5  
OPN_SV_ALL_CNT = 1-5  

Dependent
OPN_LN_BAL = OPN_LN_ALL_CNT * sample from loan distribution  
OPN_SV_BAL = OPN_SV_ALL_CNT * sample from savings distribution  

DIV_YTD_AMT = open sv bal * 1%  
INT_YTD_AMT = opn ln bal * 5%  

SV_SD_BAL = 25-opn_sv_bal  
SV_SD_CNT = 1-opn_sv_cnt  

SV_CERT_BAL = opn_sv_bal - sv_sd_bal  
SV_CERT_CNT = opn_sv_cnt - sv_sd_cnt  

MAX_DAYS_DELQ_CNT = if cls_ln_cnt > 0: (0 - 90),  2% probability of 90+ else: 0  
MAX_MTHS_DELQ_CNT = max_days_delq_cnt / 30  

WO_LN_CNT = if max days delinquent>90: cls_ln_cnt * random number from loan distribution else 0  
WO_LN_BAL = wo_ln_cnt * (random number from loan distribution)  

ignoring these for now
~SV_REG_BAL = ~
~SV_REG_CNT = 1- opn_sv_all_cnt~

In [ ]:
def generate_agr_membertotal_today(membership_participation):
    Pass
    

    
#columns
CLS_LN_CNT
CLS_SV_CNT
DIV_YTD_AMT
INT_YTD_AMT
OPN_LN_ALL_CNT
OPN_LN_BAL
OPN_SV_ALL_CNT
OPN_SV_BAL
SV_CERT_BAL
SV_CERT_CNT
SV_REG_BAL
SV_REG_CNT
SV_SD_BAL
SV_SD_CNT
WO_LN_BAL
WO_LN_CNT
MAX_DAYS_DELQ_CNT
MAX_MTHS_DELQ_CNT


In [ ]:
def quesadilla_cost_calculator(cheese_cost, slices_in_pack, tortilla_cost, tortillas, slices_per_quesadilla, tortillas_per_quesadilla):
    cost_per_slice = cheese_cost/slices_in_pack
    cost_per_tortilla= tortilla_cost/tortillas
    cost_per_quesadilla = (tortillas_per_quesadilla * cost_per_tortilla) + (slices_per_quesadilla * cost_per_slice)
    return(f'using {slices_per_quesadilla} slice of cheese and {tortillas_per_quesadilla} tortillas each, it is costing you {cost_per_quesadilla} per quesadilla')

In [ ]:
quesadilla_cost_calculator(8, 42, 4, 80, 1, 2)